# Example 3

The purpose of this example is to further illustrate implementation of large (LP) problems in Pyomo, and the separation of problem specification and problem data.

The problem to be solved is specified as:
- Consider a factory that produces $N$ products in a serial pipeline.
- For each product $n$, a profit $p_n$ is made on each unit sold, there is a weekly demand $d_n$ and a throughput of $r_n$ units per hour.
- We wish to determine the weekly production $x_n$, for each product $n$, so as to maximize the total profit in sales, given
that the factory has a total number of $T$ hours per week.

This is formulated as an LP problem in the following way:
\begin{align}
 \max \quad & \sum_{n=1}^N p_n x_n \\
 \text{subject to:} \quad & \sum_{n=1}^N \frac{1}{r_n} x_n \leq T \\
 & 0 \leq x_n \leq d_n, \quad n = 1,\ldots,N
\end{align}

We have (potentially) a large number of products, so the parameters $p_n$, $r_n$ and $d_n$ for each product is specified in a data file. The data file is specified below. But first we implement the model:

In [3]:
from pyomo.environ import *

# create model
m = AbstractModel()

# Parameters
m.T = Param() # Total number of hours available in a week

# Set
m.Products = Set()

m.p = Param(m.Products)
m.r = Param(m.Products)
m.d = Param(m.Products)

# Variables
m.x = Var(m.Products,domain=NonNegativeReals)

# Objective
def objective_rule(m):
    return summation(m.p, m.x)
m.obj = Objective(rule = objective_rule, sense=maximize)

# Constraints
def available_working_hours(m):
    return sum((1/m.r[n])*m.x[n] for n in m.Products) <= m.T
m.availworkhours = Constraint(rule = available_working_hours)

def upper_bound(m,n):
    return m.x[n] <= m.d[n]
m.upperbound = Constraint(m.Products,rule=upper_bound)

Note how we are not specifying any data, only that the different products are in the set `Products`, and the data for each product are in the `p`, `r` and `d` parameters.

We use Pyomo's [Data Command Files](https://pyomo.readthedocs.io/en/stable/working_abstractmodels/data/datfiles.html) to load the data for the problem, and import it using Pyomo's [DataPortal](https://pyomo.readthedocs.io/en/stable/working_abstractmodels/data/dataportals.html) class. Note that the DataPortal class can import many other file formats. See below. (I would have liked to write `param T` instead of `table T`, this seems to be a bug in Pyomo?)

In [4]:
!type Example3.dat

# Data for Example 3 (LP) 

table T := 40; # Number of working hours per week

set Products := Product1 Product2 Product3 Product4; # The set of products

# Data Matrix
param : p r d :=
Product1 1 40 1000
Product2 1.5 30 900
Product3 1 50 500
Product4 1.5 20 800
;


Next we can load the data into an instance of the problem, and solve:

In [5]:
data = DataPortal()
data.load(filename='Example3.dat')

m_instance = m.create_instance(data)

from pyomo.opt import SolverFactory
opt = SolverFactory('glpk')
results = opt.solve(m_instance)

m_instance.x.display()

x : Size=4, Index=Products
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
    Product1 :     0 :   0.0 :  None : False : False : NonNegativeReals
    Product2 :     0 : 900.0 :  None : False : False : NonNegativeReals
    Product3 :     0 : 500.0 :  None : False : False : NonNegativeReals
    Product4 :     0 :   0.0 :  None : False : False : NonNegativeReals


Alternatively, we could have solved via the command line:

Exercise: Implement the dual of this problem, and check if the solution is the same. Use the same data file.